In [99]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
from datetime import datetime, date, timedelta
import math
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt

import seaborn as sns
sns.set()
sns.set_theme()

import statistics as s 

In [100]:
# Lectura de datos
start = datetime(1900,1,1)
end = datetime.now()
data = pdr.get_data_yahoo('BTC-USD', start, end, interval='d')
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-09-17,468.174011,452.421997,465.864014,457.334015,2.105680e+07,457.334015
2014-09-18,456.859985,413.104004,456.859985,424.440002,3.448320e+07,424.440002
2014-09-19,427.834991,384.532013,424.102997,394.795990,3.791970e+07,394.795990
2014-09-20,423.295990,389.882996,394.673004,408.903992,3.686360e+07,408.903992
2014-09-21,412.425995,393.181000,408.084991,398.821014,2.658010e+07,398.821014
...,...,...,...,...,...,...
2021-11-22,59266.359375,55679.839844,58706.847656,56289.289062,3.503612e+10,56289.289062
2021-11-23,57875.515625,55632.761719,56304.554688,57569.074219,3.748580e+10,57569.074219
2021-11-24,57803.066406,55964.222656,57565.851562,56280.425781,3.663557e+10,56280.425781


In [101]:
def cagr(df):
    """
    cagr [retorna el CAGR en un DataFrame]
    [Una tasa de crecimiento anual compuesto (CAGR) mide la tasa de retorno de una inversión (como un fondo de inversión o un bono) durante un período de inversión, como 5 o 10 años]
    Args:
        df ([DataFrame]): [DataFrame obtenido al leer los datos del activo de interés]
    """
    df = df.copy()
    df = df.sort_index(ascending=True)
    df['cum_return'] = (1 + df['btc_return']).cumprod()
    total_ret = df['cum_return'][-1]

    start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    CAGR = (total_ret)**(1/period_years)-1
    return CAGR

In [102]:
# Crear variables y subset para análisis
df = data[['Adj Close']]['2017':'2021'].copy()
df = df.rename(columns={'Adj Close': 'btc_price'})
df['btc_return'] = df['btc_price'].pct_change()
df['day'] = df.index.day
df['weekday'] = df.index.dayofweek
df['month'] = df.index.month
df['year'] = df.index.year
df.head()

,btc_price,btc_return,day,weekday,month,year
Date,,,,,,
2017-01-01,998.325012,NaN,1,6,1,2017
2017-01-02,1021.750000,0.023464,2,0,1,2017
2017-01-03,1043.839966,0.021620,3,1,1,2017
2017-01-04,1154.729980,0.106233,4,2,1,2017
2017-01-05,1013.380005,-0.122410,5,3,1,2017


In [103]:

def DCA_variable(df, usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según la variación del retorno y la cantidad de btc obtenidos
   '''
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] < -0.0):
         df.at[x,'usdt_invested'] = 2*usdt*(1+np.abs(row['btc_return']))
      else:
         df.at[x, 'usdt_invested'] = usdt
   df['btc_amount'] = df_sem['usdt_invested']/df_sem['btc_price']
   return df.head(20)

In [104]:
#Estrategia aplicando DCA variable con compra mayor en caídas
df_sem = df[df['weekday']==1]
df_sem['btc_return'] = df_sem['btc_price'].pct_change()
DCA_variable(df_sem, 50)

,btc_price,btc_return,day,weekday,month,year,usdt_invested,btc_amount
Date,,,,,,,,
2017-01-03,1043.839966,NaN,3,1,1,2017,50.000000,0.047900
2017-01-10,907.679016,-0.130442,10,1,1,2017,113.044236,0.124542
2017-01-17,907.937988,0.000285,17,1,1,2017,50.000000,0.055070
2017-01-24,892.687012,-0.016797,24,1,1,2017,101.679738,0.113903
2017-01-31,970.403015,0.087059,31,1,1,2017,50.000000,0.051525
2017-02-07,1061.349976,0.093721,7,1,2,2017,50.000000,0.047110
2017-02-14,1004.549988,-0.053517,14,1,2,2017,105.351674,0.104874
2017-02-21,1115.300049,0.110248,21,1,2,2017,50.000000,0.044831
2017-02-28,1179.969971,0.057984,28,1,2,2017,50.000000,0.042374


In [105]:
def calculate_return(df):
    total_btc_amount = df_sem['btc_amount'].sum()
    total_usdt_invested = df_sem['usdt_invested'].sum()
    total_usdt_obtained = round(total_btc_amount*df['btc_price'][-1],3)
    total_return = (total_usdt_obtained/total_usdt_invested-1)*100
    cagr_value = cagr(df_sem)
    print(f'total_return: {round(total_return,2)}%')
    print(f'total_usdt_invested: {total_usdt_invested}')
    print(f'total_usdt_obtained: {total_usdt_obtained}')
    print(f'cagr: {round(cagr_value,2)}')

In [106]:
calculate_return(df_sem)

total_return: 940.43%
total_usdt_invested: 19602.70470414884
total_usdt_obtained: 203951.753
cagr: 1.27
